# Evaluate the Single-Linkage Ascendant Hierarchical Clustering

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import time

from skimage.morphology import skeletonize, binary_dilation, binary_erosion, binary_closing
from skimage import data
from skimage.util import invert

from sklearn.cluster import DBSCAN

In [ ]:
path = "D:/Documents/IODAA/Fil Rouge/Resultats/dIP_vs_dIR_linear_fixed_density_analysis/densite=5/0.22_0.88/Output/Session_1/Otsu_R/OTSU_R_022_088_0_2322.jpg"
save_img_path = "D:/Documents/IODAA/Fil Rouge/Resultats/dIP_vs_dIR_linear_fixed_density_analysis/densite=5/0.22_0.88/Analysis/skeleton.svg"
